In [29]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud 
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
%matplotlib inline 
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
cf.go_offline() 
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

pd.set_option('display.max_columns', None)


In [32]:
amazon = pd.read_csv("../Data/amazon.csv")

In [34]:
amazon.head(10)

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.000000
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.000000
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.000000
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.000000
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.000000
5,5,2Cents!,5,It's mini storage. It doesn't do anything els...,29-04-2013,588,0,0,0,0,0.0,0.000000
6,6,2K1Toaster,5,I have it in my phone and it never skips a bea...,19-10-2013,415,0,0,0,0,0.0,0.000000
7,7,"35-year Technology Consumer ""8-tracks to 802.11""",5,It's hard to believe how affordable digital ha...,07-10-2014,62,0,0,0,0,0.0,0.000000
8,8,4evryoung,5,Works in a HTC Rezound. Was running short of ...,24-03-2014,259,1,0,1,1,1.0,0.206549
9,9,53rdcard,5,"in my galaxy s4, super fast card, and am total...",10-11-2013,393,0,0,0,0,0.0,0.000000


In [36]:
amazon = amazon.sort_values("wilson_lower_bound", ascending=False)
amazon.drop('Unnamed: 0', inplace=True, axis=1)
amazon.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [49]:
def missing_values_analysis(amazon):
   def missing_values_analysis(amazon):
    na_columns_ = [col for col in amazon.columns if amazon[col].isnull().sum() > 0]
    n_miss = amazon[na_columns_].isnull().sum().sort_values(ascending=True)  # Fixed typo here
    ratio_ = (amazon[na_columns_].isnull().sum() / amazon.shape[0] * 100).sort_values(ascending=True)  # Fixed typo here
    missing_amazon = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Missing Values', 'Ratio'])
    return missing_amazon

def check_dataframe(amazon, head=5, tail=5):
    print("SHAPE".center(82,'~'))
    print('Rows: {}'.format(amazon.shape[0]))
    print('Colunms: {}'.format(amazon.shape[1]))
    print("TYPES".center(82,'~'))
    print(amazon.dtypes)
    print("".center(82,'~'))
    print(missing_values_analysis(amazon))
    print('DUPLICATED VALUES'.center(83,'~'))
    print(amazon.duplicated().sum())
    print("QUANTILES".center(83, '~'))
    print(amazon.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_dataframe(amazon)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rows: 4915
Colunms: 11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
None
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATED VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~QUANTILES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


TypeError: unsupported operand type(s) for -: 'str' and 'str'